In [1]:
import json
import pandas as pd

In [2]:
config_dict = json.load(open('automl_config.json','r'))

In [3]:
train = pd.read_pickle(config_dict['train_data_url'])
test = pd.read_pickle(config_dict['test_data_url'])
y_test = np.load('/home/kai/data/shiyi/data/flight_data/testY_100k.npy')

In [67]:
train_ori = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/train_ori_1M.pkl')
test_ori = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/testX_ori_100k.pkl')

In [4]:
train.shape, test.shape, y_test.shape

((1000000, 9), (100000, 8), (100000,))

In [5]:
for col in train.columns:
    print(col, train[col].nunique())

Month 12
DayofMonth 31
DayOfWeek 7
DepTime 27
UniqueCarrier 22
Origin 291
Dest 292
Distance 10
label 2


In [6]:
features_cols = list(test.columns)
features_cols

['Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'UniqueCarrier',
 'Origin',
 'Dest',
 'Distance']

In [7]:
X_train_ordinal = train[features_cols]
y_train = train['label']
X_test_ordinal = test

In [8]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import BaggingClassifier

In [9]:
l = LogisticRegression()
r = RandomForestClassifier(n_estimators=25,max_depth=10)

In [18]:
l.fit(X_train_ordinal,y_train)
y_pred = l.predict_proba(X_test_ordinal)[:,1]
log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred)

(0.48960758075351435, 0.6838084961007302)

In [19]:
%%time
r.fit(X_train_ordinal,y_train)
y_pred = r.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.47396603577325025 0.7126831724738378
CPU times: user 2min 10s, sys: 5.45 s, total: 2min 16s
Wall time: 24 s


%%time
lsvc = CalibratedClassifierCV(LinearSVC())
b_lsvc = BaggingClassifier(lsvc, max_samples=0.2, n_estimators=5, n_jobs=5)
b_lsvc.fit(X_train_ordinal, y_train)
y_pred = b_lsvc.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

In [29]:
%%time
b_l = BaggingClassifier(l, max_samples=0.8, n_estimators=4, n_jobs=1)
b_l.fit(X_train_ordinal, y_train)
y_pred = b_l.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4896134920066158 0.6838345937526396
CPU times: user 1min 34s, sys: 2.06 s, total: 1min 36s
Wall time: 1min 10s


In [25]:
%%time
b_l = BaggingClassifier(l, max_samples=0.1, n_estimators=4, n_jobs=1)
b_l.fit(X_train_ordinal, y_train)
y_pred = b_l.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4897041255472234 0.683711434908268
CPU times: user 1min 16s, sys: 1.73 s, total: 1min 17s
Wall time: 51.1 s


In [26]:
%%time
b_l = BaggingClassifier(l, max_samples=0.01, n_estimators=4, n_jobs=1)
b_l.fit(X_train_ordinal, y_train)
y_pred = b_l.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.49012944248742135 0.6836362303517354
CPU times: user 1min 5s, sys: 1.69 s, total: 1min 6s
Wall time: 41.8 s


In [27]:
%%time
b_l = BaggingClassifier(l, max_samples=0.001, n_estimators=4, n_jobs=1)
b_l.fit(X_train_ordinal, y_train)
y_pred = b_l.predict_proba(X_test_ordinal)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.49204664448401303 0.6770829948726478
CPU times: user 1min 3s, sys: 1.48 s, total: 1min 5s
Wall time: 40.8 s


## One Hot Encoding

In [10]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train_ordinal)
X_train_one_hot = enc.transform(X_train_ordinal)
X_test_one_hot = enc.transform(X_test_ordinal)

In [31]:
X_train_one_hot.shape, X_test_one_hot.shape

((1000000, 692), (100000, 692))

In [36]:
%%time
l.fit(X_train_one_hot,y_train)
y_pred = l.predict_proba(X_test_one_hot)[:,1]

CPU times: user 17.8 s, sys: 243 ms, total: 18 s
Wall time: 13.9 s


In [37]:
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.46664126300054143 0.724487672772645


In [38]:
%%time
r.fit(X_train_one_hot,y_train)
y_pred = r.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4980428696133663 0.7104962827869701
CPU times: user 1min 56s, sys: 5.28 s, total: 2min 1s
Wall time: 12.9 s


In [39]:
X_train.head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,7,19,2,18,21,157,133,6
1,4,3,1,12,18,79,172,4


In [32]:
%%time
b_l = BaggingClassifier(l, max_samples=0.1, n_estimators=8, max_features=0.4, n_jobs=1)
b_l.fit(X_train_one_hot, y_train)
y_pred = b_l.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.484711249461318 0.713394864058001
CPU times: user 2min 57s, sys: 2.83 s, total: 3min
Wall time: 2min 16s


In [37]:
%%time
b_l = BaggingClassifier(l, max_samples=0.6, n_estimators=8, n_jobs=1)
b_l.fit(X_train_one_hot, y_train)
y_pred = b_l.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4666838788887838 0.7243254170253246
CPU times: user 7min 48s, sys: 2.99 s, total: 7min 51s
Wall time: 7min 8s


In [33]:
%%time
b_r = BaggingClassifier(r, max_samples=0.1, n_estimators=8, max_features=0.4, n_jobs=1)
b_r.fit(X_train_one_hot, y_train)
y_pred = b_r.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.5076819598091701 0.7047837027229431
CPU times: user 12min 35s, sys: 42.7 s, total: 13min 18s
Wall time: 1min 4s


In [36]:
%%time
b_r = BaggingClassifier(r, max_samples=0.6, n_estimators=8, n_jobs=1)
b_r.fit(X_train_one_hot, y_train)
y_pred = b_r.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4989549307472209 0.7168805719707757
CPU times: user 16min 36s, sys: 48.8 s, total: 17min 25s
Wall time: 1min 57s


In [11]:
import lightgbm as lgb

In [179]:
features_cols

array(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance'], dtype=object)

In [20]:
lgb_params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'num_boost_round': 100, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                           # seems to work fine as the upper limit when combined with early_stopping_round
    'learning_rate': 0.05,
    'num_leaves': 15,
    'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
    'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
    'min_data_in_leaf': 20,
    'feature_fraction': 1,
    'feature_fraction_seed': 2018,
    'early_stopping_round':10,
    'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
    'bagging_fraction': 0.6,
    'bagging_seed': 2018,
    'scale_pos_weight': 1,
    'metric': 'auc' 
}

In [15]:
# cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'DepTime']
cat_cols = features_cols

In [16]:
cat_cols

['Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'UniqueCarrier',
 'Origin',
 'Dest',
 'Distance']

In [17]:
X_train_ordinal_catonly = X_train_ordinal[cat_cols]

In [12]:
from sklearn.model_selection import train_test_split

In [18]:
X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_ordinal_catonly, y_train, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=cat_cols)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=cat_cols)

In [21]:
lgbm = lgb.train(lgb_params, lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=1)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's auc: 0.694386
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.698906
[3]	valid_0's auc: 0.701591
[4]	valid_0's auc: 0.704786
[5]	valid_0's auc: 0.706526
[6]	valid_0's auc: 0.708132
[7]	valid_0's auc: 0.710976
[8]	valid_0's auc: 0.711848
[9]	valid_0's auc: 0.712659
[10]	valid_0's auc: 0.713163
[11]	valid_0's auc: 0.713512
[12]	valid_0's auc: 0.713891
[13]	valid_0's auc: 0.714354
[14]	valid_0's auc: 0.715
[15]	valid_0's auc: 0.71543
[16]	valid_0's auc: 0.715896
[17]	valid_0's auc: 0.716297
[18]	valid_0's auc: 0.716472
[19]	valid_0's auc: 0.716793
[20]	valid_0's auc: 0.717376
[21]	valid_0's auc: 0.718013
[22]	valid_0's auc: 0.718391
[23]	valid_0's auc: 0.71881
[24]	valid_0's auc: 0.719192
[25]	valid_0's auc: 0.719598
[26]	valid_0's auc: 0.719897
[27]	valid_0's auc: 0.720773
[28]	valid_0's auc: 0.721241
[29]	valid_0's auc: 0.721603
[30]	valid_0's auc: 0.721979
[31]	valid_0's auc: 0.722321
[32]	valid_0's auc: 0.722542
[33]	valid_0's auc: 0.72

In [220]:
X_train_ordinal_catonly.dtypes

Month            int64
DayofMonth       int64
DayOfWeek        int64
DepTime          int64
UniqueCarrier    int64
Origin           int64
Dest             int64
Distance         uint8
dtype: object

In [221]:
lgb_cat_feat = 'name:Month'

In [224]:
X_train_ordinal_catonly.columns

Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance'],
      dtype='object')

In [26]:
X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_ordinal_catonly, y_train, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=['Month'])
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=['Month'])

In [27]:
lgbm = lgb.train(lgb_params, lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=1)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's auc: 0.691007
[2]	valid_0's auc: 0.693526
[3]	valid_0's auc: 0.69536
[4]	valid_0's auc: 0.695687
[5]	valid_0's auc: 0.696532
[6]	valid_0's auc: 0.696906
[7]	valid_0's auc: 0.697806
[8]	valid_0's auc: 0.698039
[9]	valid_0's auc: 0.698247
[10]	valid_0's auc: 0.698526
[11]	valid_0's auc: 0.698485
[12]	valid_0's auc: 0.698464
[13]	valid_0's auc: 0.698617
[14]	valid_0's auc: 0.698979
[15]	valid_0's auc: 0.699207
[16]	valid_0's auc: 0.699607
[17]	valid_0's auc: 0.699828
[18]	valid_0's auc: 0.700226
[19]	valid_0's auc: 0.700467
[20]	valid_0's auc: 0.70065
[21]	valid_0's auc: 0.701072
[22]	valid_0's auc: 0.701214
[23]	valid_0's auc: 0.701448
[24]	valid_0's auc: 0.701614
[25]	valid_0's auc: 0.701759
[26]	valid_0's auc: 0.702317
[27]	valid_0's auc: 0.702417
[28]	valid_0's auc: 0.702896
[29]	valid_0's auc: 0.702963
[30]	valid_0's auc: 0.703338
[31]	valid_0's auc: 0.703581
[32]	valid_0's auc: 0.703812
[33]	valid_0's auc: 0.704126
[34]	valid_0's auc: 0.704329
[35]	valid_0's auc: 0.704

In [178]:
sorted(dict(zip(lgbm.feature_name(), lgbm.feature_importance())).items(), key=lambda x: x[1], reverse=True)

[('Origin', 369),
 ('Dest', 295),
 ('DepTime', 281),
 ('Month', 134),
 ('DayofMonth', 124),
 ('UniqueCarrier', 119),
 ('DayOfWeek', 78)]

In [163]:
sorted(dict(zip(lgbm.feature_name(), lgbm.feature_importance())).items(), key=lambda x: x[1], reverse=True)

[('Origin', 365),
 ('Dest', 297),
 ('DepTime', 288),
 ('Month', 128),
 ('DayofMonth', 120),
 ('UniqueCarrier', 116),
 ('DayOfWeek', 69),
 ('Distance', 17)]

In [215]:
X_train_1hot, X_val_1hot, y_train_1hot, y_val_1hot = train_test_split(X_train_one_hot, y_train, test_size=0.3, random_state=2018)
lgb_train_1hot = lgb.Dataset(X_train_1hot, y_train_1hot)
lgb_val_1hot = lgb.Dataset(X_val_1hot, y_val_1hot)

In [214]:
X_train_1hot.shape, X_val_1hot.shape, y_train_1hot.shape, y_val_1hot.shape

((700000, 692), (300000, 692), (700000,), (300000,))

In [216]:
lgbm = lgb.train(lgb_params, lgb_train_1hot, valid_sets=lgb_val_1hot, verbose_eval=1)

[1]	valid_0's auc: 0.676653
[2]	valid_0's auc: 0.691974
[3]	valid_0's auc: 0.693259
[4]	valid_0's auc: 0.693034
[5]	valid_0's auc: 0.694081
[6]	valid_0's auc: 0.695603
[7]	valid_0's auc: 0.696913
[8]	valid_0's auc: 0.697448
[9]	valid_0's auc: 0.697624
[10]	valid_0's auc: 0.697187
[11]	valid_0's auc: 0.697989
[12]	valid_0's auc: 0.697886
[13]	valid_0's auc: 0.699031
[14]	valid_0's auc: 0.698635
[15]	valid_0's auc: 0.69891
[16]	valid_0's auc: 0.698878
[17]	valid_0's auc: 0.700356
[18]	valid_0's auc: 0.701034
[19]	valid_0's auc: 0.702088
[20]	valid_0's auc: 0.702255
[21]	valid_0's auc: 0.702721
[22]	valid_0's auc: 0.70351
[23]	valid_0's auc: 0.704094
[24]	valid_0's auc: 0.704201
[25]	valid_0's auc: 0.705279
[26]	valid_0's auc: 0.705149
[27]	valid_0's auc: 0.705727
[28]	valid_0's auc: 0.705723
[29]	valid_0's auc: 0.70622
[30]	valid_0's auc: 0.70677
[31]	valid_0's auc: 0.707479
[32]	valid_0's auc: 0.708342
[33]	valid_0's auc: 0.708215
[34]	valid_0's auc: 0.708654
[35]	valid_0's auc: 0.70911